In [2]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Wed Dec  7 10:05:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   36C    P0    38W / 300W |      2MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

## LA dataset

In [ ]:
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0 import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'asv2019' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)

#dl.get_data(data_pick = '2', tde_pick = 't', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'd', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'e', pl_pick = 'l', to = 'd')


datapick = '2' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', pl_pick = 'l', to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', pl_pick = 'l', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', pl_pick = 'l', to = 'e')

#track1 = data_loader(pathset)
#track1_generator = DataGenerator(track1.eval, track1.labels, **params_no_shuffle)

#track1 = data_loader(pathset)
#track1_generator = DataGenerator(track1.eval, track1.labels, **params_no_shuffle)

## hyperparameter search

## cut_hp : cutout, cutmix hp 탐색
cut_hps = [round(i,2) for i in np.arange(0.1,1,0.1)]


## spec_hp : specaug, specmix hp 탐색
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]
#################################################
### get_ResMax
mname = 'ResMax_LA_'
#################################################

# ### 1.Cutout
# for cut_hp in cut_hps :
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = cut_hp
#         cutmix = False
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' : specmix,
#             'data_dir': asv2019,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': asv2019
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_ResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


# ### 2. CutMix  
# for cut_hp in cut_hps :       
#     for i in range(3) :
#         sr = 16000
#         sec = 9.0
#         batch_size = 16
#         feature = "cqt"
#         #hop_length = 128
#         #win_length = 512
#         #n_bins = 80
#         filter_scale = 1
#         n_bins = 100
#         fmin = 5
#         epoch = 70
#         beta_param = False 
#         cutout = False
#         cutmix = cut_hp
#         specaug = False
#         specmix = False
#         dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
#         human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
#         win_size = np.random.choice([10,15,20])
#         if win_size == 10 :
#             lpsize = [8,9,10,11,12]
#             hpsize = [88,89,90,91,92]
#             ransize = [8,9,10,11,12]
#         elif win_size == 15 :
#             lpsize = [13,14,15,16,17]
#             hpsize = [83,84,85,86,87]
#             ransize = [13,14,15,16,17]
#         else :
#             lpsize = [18,19,20,21,22]
#             hpsize = [78,79,80,81,82]
#             ransize = [18,19,20,21,22]
#         ru = np.random.uniform(.1, .9)
#         uv = [ru, 1-ru]
#         lowpass = False #np.random.choice([[uv ,lpsize], False])
#         highpass = False #np.random.choice([[uv, hpsize], False])
#         nm = np.random.choice([2,3,4])
#         ranfilter2 = False # np.random.choice([[nm, ransize], False])
#         tmp_string = "tmp"
#         params = {'sr': sr,
#             'batch_size': batch_size,
#             'feature': feature,
#             'n_classes': 2,
#             'sec': sec,
#             'filter_scale': filter_scale,
#             'fmin' : fmin,
#             'n_bins': int(n_bins),
#             'tofile': tmp_string,
#             'shuffle': True,
#             'beta_param': beta_param,
#             'cutmix' : cutmix, 
#             'cutout' : cutout, 
#             'specaug' : specaug, 
#             'specmix' :  specmix,
#             'data_dir': asv2019,
#             'lowpass': lowpass,
#             'highpass': highpass,
#             #          'ranfilter' : [10,11,12,13,14,15]
#             'ranfilter2' : ranfilter2 
#             #           'dropblock' : [30, 100]
#             #'device' : device
#         }
#         params_no_shuffle = {'sr': sr,
#                         'batch_size': batch_size,
#                         'feature': feature,
#                         'n_classes': 2,
#                         'sec': sec,
#                         'filter_scale': filter_scale,
#                         'fmin' : fmin,
#                         'n_bins': int(n_bins),
#                         'tofile': tmp_string,
#                         'shuffle': False,
#                         'data_dir': asv2019
#                         #'device': device
#         }
#         training_generator = DataGenerator(dl.train, dl.labels, **params)
#         validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
#         eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
#         input_shape = training_generator.get_input_shape()
#         model = get_ResMax(input_shape)
#     #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

#         EPOCHS = epoch
#         lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
#         class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
#         history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
#                                 epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
#         eer_val = evalEER(validation_generator, model)
#         eer_eval = evalEER(eval_generator, model)
#         endtxt1 = str(eer_val)[:6] + '.hdf5'
#         savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
#         savefnm = 'saved_models/' + savefnm
#         model.save(savefnm)
#         endtxt1 = str(eer_val)[:6] + '.npy'
#         fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
#         fnm = 'saved_results/' + fnm
#         sc1 = model.predict(eval_generator)
#         np.save(fnm, sc1)
#         params_save = params
#         params_save['model'] = mname
#         params_save['human_weight'] = str(human_weight)
#         params_save['dropout_rate'] = str(dropout_rate)
#         params_save['eer_val'] = str(eer_val)[:6]
#         params_save['eer_eval'] = str(eer_eval)[:6]
#         params_save['saved_model'] = savefnm
#         tnow = datetime.datetime.now()
#         params_save['tnow'] = str(tnow)
#         print(params_save)
#         fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
#         with open(fnm, 'wb') as f:
#             pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


## 3. Specaug
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = spec_hp
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_ResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
            
## 4. Specmix
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = spec_hp
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_ResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)     
        
################################
mname = "LCNN_LA_"
################################

### 1.Cutout
for cut_hp in cut_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = cut_hp
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### 2. CutMix  
for cut_hp in cut_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = cut_hp
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' :  specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


## 3. Specaug
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = spec_hp
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
    
## 4. Specmix
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = spec_hp
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)     
            



###############################################
mname = "BCResMax_LA_"
###############################################
### 1.Cutout
for cut_hp in cut_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = cut_hp
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### 2. CutMix  
for cut_hp in cut_hps :       
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = cut_hp
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' :  specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


## 3. Specaug
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = spec_hp
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
    
    
## 4. Specmix
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = spec_hp
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)     
            



###############################################
mname = "DDWSseq_LA_"
###############################################

### 1.Cutout
for cut_hp in cut_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = cut_hp
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


### 2. CutMix  
for cut_hp in cut_hps :      
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = cut_hp
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' :  specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


## 3. Specaug
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = spec_hp
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
    
## 4. Specmix
for spec_hp in spec_hps :
    for i in range(3) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = spec_hp
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)   

cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 45s 25ms/step - loss: 0.4493 - auc: 0.9181
Epoch 2/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.1044 - auc: 0.9962
Epoch 3/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0778 - auc: 0.9982
Epoch 4/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0571 - auc: 0.9988
Epoch 5/70
1586/1586 [==============================] - 39s 25ms/step - loss: 0.0599 - auc: 0.9987
Epoch 6/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0341 - auc: 0.9991 1s - 
Epoch 7/70
1586/1586 [==============================] - 39s 25ms/step - loss: 0.0352 - auc: 0.9994
Epoch 8/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0269 - auc: 0.9996
Epoch 9/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0361 - auc: 0.9989
Epoch 10/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0210 - auc: 0.9994
Epo

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.005507080426304999


...

EER: 0.451871896598414


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.44593864091617674
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': False, 'cutmix': False, 'cutout': False, 'specaug': False, 'specmix': [[2, 15], [2, 15]], 'data_dir': '/home/ubuntu/data/asv2019/', 'lowpass': False, 'highpass': False, 'ranfilter2': False, 'model': 'LCNN_LA_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.4518', 'eer_eval': '0.4459', 'saved_model': 'saved_models/model_name!LCNN_LA_!dropout_rate!na!human_weight!na!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!False!cutmix!False!cutout!False!specaug!False!specmix![[2, 15], [2, 15]]!lowpass!--!highpass!--!ranfilter2!--!0.4518.hdf5', 'tnow': '2022-08-21 06:44:15.233321'}
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 32s 19ms/step - loss: 1.2243 - auc: 0.6885
Epoch 2/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9541 - auc: 0.8733
Epoch 3/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9151 - auc: 0.9038
Epoch 4/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9217 - auc: 0.8981
Epoch 5/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9181 - auc: 0.9021
Epoch 6/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9143 - auc: 0.9027
Epoch 7/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9228 - auc: 0.8986
Epoch 8/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9203 - auc: 0.8992
Epoch 9/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9208 - auc: 0.8994
Epoch 10/70
1586/1586 [==============================] - 30s 19ms/step - loss: 0.9177 - auc: 0.9000
Epoch 11/

## filter augment  실험

In [ ]:
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0 import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'asv2019' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)



datapick = '2' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', pl_pick = 'l', to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', pl_pick = 'l', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', pl_pick = 'l', to = 'e')


## hyperparameter search

## cut_hp : cutout, cutmix hp 탐색
cut_hps = [round(i,2) for i in np.arange(0.1,1,0.1)]

# filter augment band 
filt_hps = [[2,5],[3,6],[5,8]] 

## spec_hp : specaug, specmix hp 탐색
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]



#################################################
### get_ResMax
mname = 'ResMax_LA_'
#################################################

### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "step" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_ResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
            

### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_ResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


################################
mname = "LCNN_LA_"
################################

### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "step" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


###############################################
mname = "BCResMax_LA_"
###############################################


### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "step" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)



###############################################
mname = "DDWSseq_LA_"
###############################################

### 5.filter aug -step
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "step" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(5) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 45s 25ms/step - loss: 0.6005 - auc: 0.8769
Epoch 2/70
1586/1586 [==============================] - 39s 25ms/step - loss: 0.1476 - auc: 0.9939
Epoch 3/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.1086 - auc: 0.9967
Epoch 4/70
1586/1586 [==============================] - 40s 26ms/step - loss: 0.0877 - auc: 0.9976
Epoch 5/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0720 - auc: 0.9983
Epoch 6/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0669 - auc: 0.9986
Epoch 7/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0476 - auc: 0.9988
Epoch 8/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0509 - auc: 0.9991
Epoch 9/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0290 - auc: 0.9996
Epoch 10/70
1586/1586 [==============================] - 40s 25ms/step - loss: 0.0277 - auc: 0.9996
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.007453059928250339


...

EER: 0.0069484071330367025


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.030079774765925316
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': False, 'cutmix': False, 'cutout': False, 'specaug': False, 'specmix': False, 'data_dir': '/home/ubuntu/data/asv2019/', 'lowpass': False, 'highpass': False, 'filteraug': [[5, 8], 2, 'linear', [-10, 10]], 'ranfilter2': False, 'model': 'LCNN_LA_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.0069', 'eer_eval': '0.0300', 'saved_model': 'saved_models/model_name!LCNN_LA_!dropout_rate!na!human_weight!na!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!False!cutmix!False!cutout!False!specaug!False!specmix!False!lowpass!--!highpass!--!filteraug!--!ranfilter2!--!0.0069.hdf5', 'tnow': '2022-11-10 00:44:58.383319'}
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 67s 41ms/step - loss: 0.9935 - auc: 0.7519
Epoch 2/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.2727 - auc: 0.9823
Epoch 3/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.1864 - auc: 0.9913
Epoch 4/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.1681 - auc: 0.9927
Epoch 5/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.1101 - auc: 0.9966
Epoch 6/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.1053 - auc: 0.9968
Epoch 7/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.0944 - auc: 0.9971
Epoch 8/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.0778 - auc: 0.9980
Epoch 9/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.0716 - auc: 0.9982
Epoch 10/70
1586/1586 [==============================] - 65s 41ms/step - loss: 0.0559 - auc: 0.9989
Epoch 11/

In [1]:
!nvidia-smi

Mon Nov  7 15:09:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   55C    P0   133W / 300W |  31097MiB / 32510MiB |     76%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [1]:
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0 import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'asv2019' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)



datapick = '2' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', pl_pick = 'l', to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', pl_pick = 'l', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', pl_pick = 'l', to = 'e')


## hyperparameter search

## cut_hp : cutout, cutmix hp 탐색
cut_hps = [round(i,2) for i in np.arange(0.1,1,0.1)]

# filter augment band 
filt_hps = [[2,5],[3,6],[5,8]] 

## spec_hp : specaug, specmix hp 탐색
spec_hps = [ [[1, 5], [1, 5]], [[1, 10], [1, 10]], [[1, 10], [1, 15]],[[1, 15], [1, 10]],[[1, 15], [1, 15]],
                  [[2, 5], [2, 5]], [[2, 10], [2, 10]], [[2, 10], [2, 15]],[[2, 15], [2, 10]],[[2, 15], [2, 15]] ]



#################################################
### get_ResMax
mname = 'ResMax_LA_'
#################################################

            

### 5.filter aug -linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_ResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


################################
mname = "LCNN_LA_"
################################

### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_LCNN(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


###############################################
mname = "BCResMax_LA_"
###############################################





### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_BCResMax(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)



###############################################
mname = "DDWSseq_LA_"
###############################################




### 6.filter aug - linear
for filt_hp in filt_hps :
    for i in range(10) :
        sr = 16000
        sec = 9.0
        batch_size = 16
        feature = "cqt"
        #hop_length = 128
        #win_length = 512
        #n_bins = 80
        filter_scale = 1
        n_bins = 100
        fmin = 5
        epoch = 70
        beta_param = False 
        cutout = False
        cutmix = False
        specaug = False
        specmix = False
        dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
        human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
        win_size = np.random.choice([10,15,20])
        if win_size == 10 :
            lpsize = [8,9,10,11,12]
            hpsize = [88,89,90,91,92]
            ransize = [8,9,10,11,12]
        elif win_size == 15 :
            lpsize = [13,14,15,16,17]
            hpsize = [83,84,85,86,87]
            ransize = [13,14,15,16,17]
        else :
            lpsize = [18,19,20,21,22]
            hpsize = [78,79,80,81,82]
            ransize = [18,19,20,21,22]
        ru = np.random.uniform(.1, .9)
        uv = [ru, 1-ru]
        lowpass = False #np.random.choice([[uv ,lpsize], False])
        highpass = False #np.random.choice([[uv, hpsize], False])
        nm = np.random.choice([2,3,4])
        ranfilter2 = False # np.random.choice([[nm, ransize], False])
        
        n_band2 = filt_hp #[[2,5],[3,6],[5,8]] # 저자는 [3,6]
        min_bw2 = [2,4,6,8,10] # 저자는 6
        f_type2 = "linear" #np.random.choice(["step", "linear"])
        db_range2 = [[-10,10],[-6,6],[-1,1]] # [-6,6]
        filteraug2 = [n_band2, np.random.choice(min_bw2), f_type2, db_range2[np.random.choice([0,1,2])]]
        
        tmp_string = "tmp"
        params = {'sr': sr,
            'batch_size': batch_size,
            'feature': feature,
            'n_classes': 2,
            'sec': sec,
            'filter_scale': filter_scale,
            'fmin' : fmin,
            'n_bins': int(n_bins),
            'tofile': tmp_string,
            'shuffle': True,
            'beta_param': beta_param,
            'cutmix' : cutmix, 
            'cutout' : cutout, 
            'specaug' : specaug, 
            'specmix' : specmix,
            'data_dir': asv2019,
            'lowpass': lowpass,
            'highpass': highpass,
            'filteraug': filteraug2,
            #          'ranfilter' : [10,11,12,13,14,15]
            'ranfilter2' : ranfilter2 
            #           'dropblock' : [30, 100]
            #'device' : device
        }
        params_no_shuffle = {'sr': sr,
                        'batch_size': batch_size,
                        'feature': feature,
                        'n_classes': 2,
                        'sec': sec,
                        'filter_scale': filter_scale,
                        'fmin' : fmin,
                        'n_bins': int(n_bins),
                        'tofile': tmp_string,
                        'shuffle': False,
                        'data_dir': asv2019
                        #'device': device
        }
        training_generator = DataGenerator(dl.train, dl.labels, **params)
        validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
        eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
        input_shape = training_generator.get_input_shape()
        model = get_DDWSseq(input_shape)
    #    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

        EPOCHS = epoch
        lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
        class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
        history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                                epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
        eer_val = evalEER(validation_generator, model)
        eer_eval = evalEER(eval_generator, model)
        endtxt1 = str(eer_val)[:6] + '.hdf5'
        savefnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1 )
        savefnm = 'saved_models/' + savefnm
        model.save(savefnm)
        endtxt1 = str(eer_val)[:6] + '.npy'
        fnm = gen_fname(model_name = mname, params = params ,  endtxt = endtxt1)
        fnm = 'saved_results/' + fnm
        sc1 = model.predict(eval_generator)
        np.save(fnm, sc1)
        params_save = params
        params_save['model'] = mname
        params_save['human_weight'] = str(human_weight)
        params_save['dropout_rate'] = str(dropout_rate)
        params_save['eer_val'] = str(eer_val)[:6]
        params_save['eer_eval'] = str(eer_eval)[:6]
        params_save['saved_model'] = savefnm
        tnow = datetime.datetime.now()
        params_save['tnow'] = str(tnow)
        print(params_save)
        fnm = 'res_mask_LA_ab_hp/rec'+str(tnow) + '.pickle'
        with open(fnm, 'wb') as f:
            pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 65s 38ms/step - loss: 0.5625 - auc: 0.8882
Epoch 2/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.1258 - auc: 0.9954
Epoch 3/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.0857 - auc: 0.9978
Epoch 4/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.0607 - auc: 0.9988
Epoch 5/70
1586/1586 [==============================] - 60s 38ms/step - loss: 0.0522 - auc: 0.9987
Epoch 6/70
1586/1586 [==============================] - 60s 38ms/step - loss: 0.0444 - auc: 0.9992
Epoch 7/70
1586/1586 [==============================] - 60s 38ms/step - loss: 0.0432 - auc: 0.9995
Epoch 8/70
1586/1586 [==============================] - 60s 38ms/step - loss: 0.0348 - auc: 0.9994
Epoch 9/70
1586/1586 [==============================] - 60s 38ms/step - loss: 0.0355 - auc: 0.9993
Epoch 10/70
1586/1586 [==============================] - 60s 38ms/step - loss: 0.0213 - auc: 0.9997
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.005507080426304999


...

Epoch 1/70
1586/1586 [==============================] - 70s 40ms/step - loss: 1.3980 - auc: 0.7214
Epoch 2/70
1586/1586 [==============================] - 63s 40ms/step - loss: 0.8671 - auc: 0.8912
Epoch 3/70
1586/1586 [==============================] - 63s 39ms/step - loss: 0.3853 - auc: 0.9709
Epoch 4/70
1586/1586 [==============================] - 62s 39ms/step - loss: 0.2887 - auc: 0.9833
Epoch 5/70
1586/1586 [==============================] - 62s 39ms/step - loss: 0.2176 - auc: 0.9897
Epoch 6/70
1586/1586 [==============================] - 62s 39ms/step - loss: 0.1811 - auc: 0.9917
Epoch 7/70
1586/1586 [==============================] - 63s 40ms/step - loss: 0.1605 - auc: 0.9938
Epoch 8/70
1586/1586 [==============================] - 62s 39ms/step - loss: 0.1609 - auc: 0.9940
Epoch 9/70
1586/1586 [==============================] - 62s 39ms/step - loss: 0.1314 - auc: 0.9956
Epoch 10/70
1586/1586 [==============================] - 63s 40ms/step - loss: 0.1214 - auc: 0.9961
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.007122201848719261


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.03289319517407418
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': False, 'cutmix': False, 'cutout': False, 'specaug': False, 'specmix': False, 'data_dir': '/home/ubuntu/data/asv2019/', 'lowpass': False, 'highpass': False, 'filteraug': [[5, 8], 4, 'linear', [-6, 6]], 'ranfilter2': False, 'model': 'DDWSseq_LA_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.0071', 'eer_eval': '0.0328', 'saved_model': 'saved_models/model_name!DDWSseq_LA_!dropout_rate!na!human_weight!na!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!False!cutmix!False!cutout!False!specaug!False!specmix!False!lowpass!--!highpass!--!filteraug!--!ranfilter2!--!0.0071.hdf5', 'tnow': '2022-11-29 10:54:22.611260'}
